In [39]:
import os

import argparse
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [41]:
RES_PATH = "../res"
DOCS_PATH = os.path.join(RES_PATH, "docs")
WELLNESS_PATH = os.path.join(DOCS_PATH, "Bienestar")
PHYSICAL_HEALTH_PATH = os.path.join(DOCS_PATH, "Salud Fisica")
MENTAL_HEALTH_PATH = os.path.join(DOCS_PATH, "Salud Mental")
ALL_DOCS = os.path.join(DOCS_PATH, "All_docs")

CHROMA_PATH = os.path.join(RES_PATH, "CHROMA_DB")
DATA_PATH = "data"

openai_keys_path = os.path.join(RES_PATH, os.path.join("keys", "openai_key.txt"))


In [3]:
def load_documents(docs_path: str):
    document_loader = PyPDFDirectoryLoader(docs_path)
    return document_loader.load()

In [ ]:
%%capture
documents = load_documents(ALL_DOCS)

In [10]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [22]:
chunks = split_documents(documents)

In [32]:
def get_embedding_function():
    embeddings = OllamaEmbeddings( model="nomic-embed-text" )
    return embeddings

In [29]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [37]:
# def add_to_chroma(chunks: list[Document]):
#     # Load the existing database.
#     db = Chroma(
#         persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
#     )

#     db.add_documents(chunks)

#     # Calculate Page IDs.
#     chunks_with_ids = calculate_chunk_ids(chunks)

#     # Add or Update the documents.
#     existing_items = db.get(include=[])  # IDs are always included by default
#     existing_ids = set(existing_items["ids"])
#     print(f"Number of existing documents in DB: {len(existing_ids)}")

#     # Only add documents that don't exist in the DB.
#     new_chunks = []
#     for chunk in chunks_with_ids:
#         if chunk.metadata["id"] not in existing_ids:
#             new_chunks.append(chunk)

#     if len(new_chunks):
#         print(f"👉 Adding new documents: {len(new_chunks)}")
#         new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
#         db.add_documents(new_chunks, ids=new_chunk_ids)
#         db.persist()
#     else:
#         print("✅ No new documents to add")

In [42]:
def save_to_chroma(chunks: list[Document]):

    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(openai_api_key=), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")

In [43]:
save_to_chroma(chunks)

c:\Users\inigo\anaconda3\envs\HPE\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)